# 取得考慮除權息的前收盤價 (previousClose / referencePrice) 演示

本演示筆記本展示如何取得歷史任意日某股票考慮除權息調整的前收盤價。

## 核心概念

- **previousClose**: 通常是 T-1 日收盤價，**未含**除權息調整
- **referencePrice**: 考慮除權息調整的參考價，用於漲跌幅計算
- 除權息參考價公式：
  $$\text{除權息參考價} = \frac{\text{除權息前收盤價} - \text{息值} + \text{現增價} \times \text{現增配股率}}{1 + \text{無償配股率} + \text{現增配股率}}$$

## 解決方案

1. **方法 A**: 用 `change` 回推：`adjusted_preclose = close - change`
2. **方法 B**: 使用 `historical/stats` 直接取得調整後的 `previousClose`
3. **方法 C**: 比較不同方法的差異


In [1]:
# 導入所需套件
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fugle_marketdata import RestClient
import warnings
warnings.filterwarnings('ignore')

# 載入環境變數
load_dotenv(find_dotenv())


True

In [2]:
# 初始化 Fugle API 客戶端
client = RestClient(api_key=os.getenv("FUGLE_API_KEY"))
stock = client.stock
print("Fugle API 客戶端初始化完成")


Fugle API 客戶端初始化完成


## 方法 A: 用 change 回推調整後的前收盤價

這個方法利用 `historical/candles` 的 `change` 欄位，該欄位已經考慮了除權息調整。


In [3]:
def get_adjusted_preclose_by_change(symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    方法 A: 使用 change 欄位回推調整後的前收盤價
    
    Args:
        symbol: 股票代碼
        start_date: 開始日期 (YYYY-MM-DD)
        end_date: 結束日期 (YYYY-MM-DD)
    
    Returns:
        DataFrame with columns: date, close, change, adjusted_preclose
    """
    try:
        # 取得歷史K線數據，包含 close 和 change
        candles = stock.historical.candles(
            symbol=symbol,
            **{
                "from": start_date,
                "to": end_date,
                "fields": "open,high,low,close,volume,change",
                "timeframe": "D",
                "sort": "asc"
            }
        )
        
        # 轉換為 DataFrame
        df = pd.DataFrame(candles["data"])
        df["date"] = pd.to_datetime(df["date"])
        
        # 計算調整後的前收盤價
        # change 已經考慮除權息調整，所以 adjusted_preclose = close - change
        df["adjusted_preclose"] = df["close"] - df["change"]
        
        return df[["date", "close", "change", "adjusted_preclose"]].round(2)
        
    except Exception as e:
        print(f"取得數據失敗: {e}")
        return pd.DataFrame()

# 測試範例：台積電 (2330) 最近一個月
start_date = "2025-06-15"
end_date = "2025-07-15"
symbol = "2330"

df_method_a = get_adjusted_preclose_by_change(symbol, start_date, end_date)
print(f"方法 A 結果 ({symbol}): 共 {len(df_method_a)} 筆數據")
print("\n最近 5 天數據:")
print(df_method_a.tail())


方法 A 結果 (2330): 共 21 筆數據

最近 5 天數據:
         date  close  change  adjusted_preclose
16 2025-07-08   1080       0               1080
17 2025-07-09   1090      10               1080
18 2025-07-10   1100      10               1090
19 2025-07-11   1100       0               1100
20 2025-07-14   1095      -5               1100


## 方法 B: 使用 historical/stats 直接取得調整後的 previousClose

這個方法直接使用 `historical/stats` 端點，該端點會返回調整後的 `previousClose`。


In [4]:
def get_adjusted_preclose_by_stats(symbol: str, target_date: str = None) -> dict:
    """
    方法 B: 使用 historical/stats 直接取得調整後的 previousClose
    
    Args:
        symbol: 股票代碼
        target_date: 目標日期 (可選，默認為最新交易日)
    
    Returns:
        包含股票資訊的字典，包括調整後的 previousClose
    """
    try:
        # historical/stats 返回最新交易日的統計資料
        stats = stock.historical.stats(symbol=symbol)
        
        return {
            "symbol": stats["symbol"],
            "name": stats["name"],
            "date": stats["date"],
            "closePrice": stats["closePrice"],
            "previousClose": stats["previousClose"],  # 這個已經是調整後的
            "change": stats["change"],
            "tradeVolume": stats["tradeVolume"],
            "tradeValue": stats["tradeValue"]
        }
        
    except Exception as e:
        print(f"取得數據失敗: {e}")
        return {}

# 測試範例：取得不同股票的調整後前收盤價
symbols = ["2330", "0050", "2454"]  # 台積電、元大台灣50、聯發科

print("方法 B 結果 (historical/stats):")
print("-" * 80)

for symbol in symbols:
    stats = get_adjusted_preclose_by_stats(symbol)
    if stats:
        print(f"{stats['symbol']} {stats['name']}:")
        print(f"  日期: {stats['date']}")
        print(f"  收盤價: {stats['closePrice']}")
        print(f"  調整後前收盤價: {stats['previousClose']}")
        print(f"  漲跌: {stats['change']}")
        print(f"  驗證: {stats['closePrice']} - {stats['change']} = {stats['closePrice'] - stats['change']}")
        print()


方法 B 結果 (historical/stats):
--------------------------------------------------------------------------------
2330 台積電:
  日期: 2025-07-14
  收盤價: 1095
  調整後前收盤價: 1100
  漲跌: -5
  驗證: 1095 - -5 = 1100

0050 元大台灣50:
  日期: 2025-07-14
  收盤價: 49.33
  調整後前收盤價: 49.62
  漲跌: -0.29
  驗證: 49.33 - -0.29 = 49.62

2454 聯發科:
  日期: 2025-07-14
  收盤價: 1385
  調整後前收盤價: 1420
  漲跌: -35
  驗證: 1385 - -35 = 1420



## 方法 C: 比較未調整 vs 調整後的前收盤價

展示除權息日的差異，以及不同方法的一致性。


In [5]:
def compare_preclose_methods(symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    比較未調整 vs 調整後的前收盤價
    
    Args:
        symbol: 股票代碼
        start_date: 開始日期
        end_date: 結束日期
    
    Returns:
        包含比較結果的 DataFrame
    """
    try:
        # 方法 A: 用 change 回推
        df_change = get_adjusted_preclose_by_change(symbol, start_date, end_date)
        
        if df_change.empty:
            return pd.DataFrame()
        
        # 計算未調整的前收盤價 (簡單的 shift)
        df_change["naive_preclose"] = df_change["close"].shift(1)
        
        # 計算差異
        df_change["difference"] = df_change["naive_preclose"] - df_change["adjusted_preclose"]
        df_change["has_ex_dividend"] = abs(df_change["difference"]) > 0.01  # 差異超過 0.01 元視為除權息
        
        # 重新排列欄位
        result = df_change[["date", "close", "naive_preclose", "adjusted_preclose", 
                           "difference", "change", "has_ex_dividend"]].round(2)
        
        return result
        
    except Exception as e:
        print(f"比較失敗: {e}")
        return pd.DataFrame()

# 測試比較功能
print("方法 C: 比較未調整 vs 調整後的前收盤價")
print("=" * 80)

# 使用台積電作為範例，因為大型股可能有除權息
comparison_df = compare_preclose_methods("2330", "2025-06-01", "2025-07-15")

if not comparison_df.empty:
    print(f"\n台積電 (2330) 比較結果:")
    print("\n所有數據:")
    print(comparison_df)
    
    # 檢查是否有除權息日
    ex_dividend_days = comparison_df[comparison_df["has_ex_dividend"]]
    if not ex_dividend_days.empty:
        print(f"\n⚠️ 發現可能的除權息日 (差異 > 0.01):")
        print(ex_dividend_days[["date", "naive_preclose", "adjusted_preclose", "difference"]])
    else:
        print("\n✅ 此期間內未發現明顯的除權息調整")
        
    # 顯示統計
    print(f"\n統計摘要:")
    print(f"  總交易日數: {len(comparison_df)}")
    print(f"  平均差異: {comparison_df['difference'].mean():.4f}")
    print(f"  最大差異: {comparison_df['difference'].max():.4f}")
    print(f"  最小差異: {comparison_df['difference'].min():.4f}")
else:
    print("無法取得比較數據")


方法 C: 比較未調整 vs 調整後的前收盤價

台積電 (2330) 比較結果:

所有數據:
         date  close  naive_preclose  adjusted_preclose  difference  change  \
0  2025-06-02    946             NaN                967         NaN     -21   
1  2025-06-03    950           946.0                946         0.0       4   
2  2025-06-04    990           950.0                950         0.0      40   
3  2025-06-05    998           990.0                990         0.0       8   
4  2025-06-06    995           998.0                998         0.0      -3   
5  2025-06-09   1005           995.0                995         0.0      10   
6  2025-06-10   1045          1005.0               1005         0.0      40   
7  2025-06-11   1065          1045.0               1045         0.0      20   
8  2025-06-12   1045          1065.0               1045        20.0       0   
9  2025-06-13   1030          1045.0               1045         0.0     -15   
10 2025-06-16   1025          1030.0               1030         0.0      -5   
11 

## 實用函數: 取得特定日期的調整後前收盤價

建立一個通用函數，可以查詢任意股票任意日期的調整後前收盤價。


In [6]:
def get_reference_price(symbol: str, target_date: str) -> dict:
    """
    取得特定日期的調整後前收盤價 (referencePrice)
    
    Args:
        symbol: 股票代碼
        target_date: 目標日期 (YYYY-MM-DD)
    
    Returns:
        包含該日期調整後前收盤價的字典
    """
    try:
        # 計算查詢範圍 (向前多取幾天以確保能找到數據)
        target_dt = datetime.strptime(target_date, "%Y-%m-%d")
        start_dt = target_dt - timedelta(days=10)
        end_dt = target_dt + timedelta(days=2)
        
        start_date = start_dt.strftime("%Y-%m-%d")
        end_date = end_dt.strftime("%Y-%m-%d")
        
        # 取得數據
        df = get_adjusted_preclose_by_change(symbol, start_date, end_date)
        
        if df.empty:
            return {"error": "無法取得數據"}
        
        # 找到目標日期的數據
        target_data = df[df["date"].dt.strftime("%Y-%m-%d") == target_date]
        
        if target_data.empty:
            # 如果目標日期沒有數據，找最接近的交易日
            df["date_str"] = df["date"].dt.strftime("%Y-%m-%d")
            available_dates = df["date_str"].tolist()
            closest_date = min(available_dates, key=lambda x: abs(
                datetime.strptime(x, "%Y-%m-%d") - target_dt
            ))
            target_data = df[df["date_str"] == closest_date]
            
            result = {
                "symbol": symbol,
                "requested_date": target_date,
                "actual_date": closest_date,
                "note": f"目標日期 {target_date} 非交易日，使用最接近的交易日",
                "close": float(target_data.iloc[0]["close"]),
                "adjusted_preclose": float(target_data.iloc[0]["adjusted_preclose"]),
                "change": float(target_data.iloc[0]["change"])
            }
        else:
            result = {
                "symbol": symbol,
                "date": target_date,
                "close": float(target_data.iloc[0]["close"]),
                "adjusted_preclose": float(target_data.iloc[0]["adjusted_preclose"]),
                "change": float(target_data.iloc[0]["change"])
            }
        
        return result
        
    except Exception as e:
        return {"error": f"查詢失敗: {e}"}

# 測試實用函數
print("實用函數測試:")
print("=" * 60)

test_cases = [
    ("2330", "2025-07-14"),  # 台積電 - 週日 (假日)
    ("0050", "2025-07-15"),  # 元大台灣50 - 週一
    ("2454", "2025-07-11"),  # 聯發科 - 週四
]

for symbol, date in test_cases:
    print(f"\n查詢 {symbol} 在 {date} 的調整後前收盤價:")
    result = get_reference_price(symbol, date)
    
    if "error" in result:
        print(f"  ❌ {result['error']}")
    else:
        if "note" in result:
            print(f"  📅 {result['note']}")
            print(f"  實際日期: {result['actual_date']}")
        else:
            print(f"  📅 交易日期: {result['date']}")
            
        print(f"  💰 當日收盤價: {result['close']}")
        print(f"  📊 調整後前收盤價: {result['adjusted_preclose']}")
        print(f"  📈 漲跌: {result['change']}")


實用函數測試:

查詢 2330 在 2025-07-14 的調整後前收盤價:
  📅 交易日期: 2025-07-14
  💰 當日收盤價: 1095.0
  📊 調整後前收盤價: 1100.0
  📈 漲跌: -5.0

查詢 0050 在 2025-07-15 的調整後前收盤價:
  📅 目標日期 2025-07-15 非交易日，使用最接近的交易日
  實際日期: 2025-07-14
  💰 當日收盤價: 49.33
  📊 調整後前收盤價: 49.62
  📈 漲跌: -0.29

查詢 2454 在 2025-07-11 的調整後前收盤價:
  📅 交易日期: 2025-07-11
  💰 當日收盤價: 1420.0
  📊 調整後前收盤價: 1400.0
  📈 漲跌: 20.0


## 總結與使用建議

### 方法選擇指南

| 使用情境 | 推薦方法 | 原因 |
|---------|---------|------|
| **日內交易監控** | 方法 A (`change` 回推) | 可以取得歷史任意日期，適合建立時間序列 |
| **即時行情分析** | 方法 B (`historical/stats`) | 直接提供最新的調整後數據，最簡單 |
| **回測系統** | 方法 A | 需要完整的歷史時間序列 |
| **除權息日檢測** | 方法 C (比較分析) | 可以識別除權息日及其影響 |

### 重要提醒

1. **除權息調整的重要性**:
   - 未調整的前收盤價在除權息日會造成「假跳空」
   - 調整後的前收盤價才是正確的漲跌幅基準

2. **數據一致性**:
   - Fugle API 的 `change` 欄位已經考慮除權息調整
   - `previousClose = close - change` 是可靠的計算方式

3. **實務應用**:
   - 製作漲跌停板監控系統時，務必使用調整後的前收盤價
   - 計算日報酬率時，建議使用還原權值的價格序列

### API 使用限制

- `historical/candles`: 可查詢歷史區間，但需注意 API 配額
- `historical/stats`: 僅返回最新交易日數據，但包含完整統計資訊
- 建議在生產環境中實施快取機制以節省 API 呼叫次數


In [7]:
# 完整示例：建立一個簡單的股價監控函數
def stock_monitor(symbol: str, alert_threshold: float = 0.05):
    """
    股價監控函數示例
    
    Args:
        symbol: 股票代碼
        alert_threshold: 警報閾值 (漲跌幅超過此比例時發出警報)
    """
    try:
        # 取得最新數據
        stats = get_adjusted_preclose_by_stats(symbol)
        
        if not stats:
            print(f"❌ 無法取得 {symbol} 的數據")
            return
        
        # 計算漲跌幅
        change_pct = stats["change"] / stats["previousClose"] * 100
        
        # 判斷是否需要警報
        is_alert = abs(change_pct) >= alert_threshold * 100
        
        print(f"📊 {stats['symbol']} {stats['name']} 監控報告")
        print(f"📅 日期: {stats['date']}")
        print(f"💰 收盤價: {stats['closePrice']}")
        print(f"📈 調整後前收盤價: {stats['previousClose']}")
        print(f"📊 漲跌: {stats['change']} ({change_pct:+.2f}%)")
        print(f"📈 成交量: {stats['tradeVolume']:,}")
        print(f"💵 成交值: {stats['tradeValue']:,}")
        
        if is_alert:
            print(f"🚨 警報: 漲跌幅 {change_pct:+.2f}% 超過閾值 {alert_threshold*100}%")
        else:
            print(f"✅ 正常: 漲跌幅在預期範圍內")
            
    except Exception as e:
        print(f"❌ 監控失敗: {e}")

# 示例：監控幾支熱門股票
print("股價監控示例:")
print("=" * 80)

watch_list = [
    ("2330", 0.03),  # 台積電，3% 警報閾值
    ("0050", 0.02),  # 元大台灣50，2% 警報閾值  
    ("2454", 0.05),  # 聯發科，5% 警報閾值
]

for symbol, threshold in watch_list:
    print()
    stock_monitor(symbol, threshold)
    print("-" * 60)


股價監控示例:

📊 2330 台積電 監控報告
📅 日期: 2025-07-14
💰 收盤價: 1095
📈 調整後前收盤價: 1100
📊 漲跌: -5 (-0.45%)
📈 成交量: 29,975,913
💵 成交值: 32,777,573,059
✅ 正常: 漲跌幅在預期範圍內
------------------------------------------------------------

📊 0050 元大台灣50 監控報告
📅 日期: 2025-07-14
💰 收盤價: 49.33
📈 調整後前收盤價: 49.62
📊 漲跌: -0.29 (-0.58%)
📈 成交量: 65,170,727
💵 成交值: 3,216,296,277
✅ 正常: 漲跌幅在預期範圍內
------------------------------------------------------------

📊 2454 聯發科 監控報告
📅 日期: 2025-07-14
💰 收盤價: 1385
📈 調整後前收盤價: 1420
📊 漲跌: -35 (-2.46%)
📈 成交量: 6,067,428
💵 成交值: 8,449,999,917
✅ 正常: 漲跌幅在預期範圍內
------------------------------------------------------------
